In [2]:
import pandas as pd
import openpyxl
from openpyxl.styles import Alignment, numbers

#月次excelデータ
file = "本社.xlsx"

#奉行から出力した月次決算excelデータ
data = "Book1月_全社.xlsx"

def honsya(file,data):
    #######################################################
    #各種データの読み込み
    #######################################################
    
    #月次excelデータ
    file = file
    wb = openpyxl.load_workbook(filename=file, read_only=False)
    ws = wb.worksheets[1]
    
    #奉行から出力した月次決算excelデータ
    data = data
    xlsx = pd.ExcelFile(data)
    sheet_names = xlsx.sheet_names
    
    #######################################################
    #読み込んだデータの整形
    #######################################################
    
    #奉行月次決算Excelデータ内の複数シートを整形
    #結合
    df = pd.concat(
        pd.read_excel(xlsx,sheet_name=sheet)
            for sheet in sheet_names)
    
    # インデックスリセット
    df.reset_index(drop=True, inplace=True)
    df = df.fillna(0)
    df = df.replace('\u3000', '', regex=True)
    
    
    #######################################################
    #メインプログラム
    #######################################################
    
    df_month = pd.DataFrame({
        "月":
        ["1月","2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月"],
        "翌月":
        ["2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月","1月"]
    })
    
    #月次Excelの更新
    for i in range(len(df_month.axes[0])):
        if df_month.iloc[i,0] == ws.cell(1,6).value :
            ws.cell(1,6).value = df_month.iloc[i,1]
            break
    
    #月次Excel(B column)の参照
    for i in range(ws.max_row):
        #奉行データ(df.axes[0]縦方向)の参照
        for j in range(len(df.axes[0])):
            #月次Excelのセル値と奉行データフレームの値が合致してれば以下の処理(for col～)
            if str(ws.cell(i+1,2).value) == df.iloc[j,0]:
                #月次Excel t～AE(1月~3月)とcell(1,6)を参照し、t~AEの列に数値を入れ込む
                for col in range(20,ws.max_column):
                    if ws.cell(3,col).value == ws.cell(1,6).value : 
                        ws.cell(i+1,col).value = df.iloc[j,2]
                        #数値の整形[カンマ、小数点なし、右詰め]
                        ws.cell(i+1,col).number_format = "#,##0"
                        ws.cell(i+1,col).alignment = Alignment(horizontal='right')
                        break
    
    
    #本社のみ適用
    #global変数の定義
    x = 0
    y = 0
    for i in range(len(df.axes[0])):
        #奉行データ内に"5131(内部仕入 堺工場 がいしG)"があるかを確認
        if df.iloc[i,0] == "5131":
            #5131内部仕入高
            x = df.iloc[i,2]
            #5120仕入高
            y = df.iloc[i-1,2]
            #月次Excel内の"材料仕入高"に合致した行を探す
            for j in range(ws.max_row):
                if str(ws.cell(j+1,2).value) == "5120":
                    #4月～3月の間で、最新の値を上書きする
                    for col in range(20,ws.max_column):
                        if ws.cell(3,col).value == ws.cell(1,6).value : 
                            ws.cell(j+1,col).value = x+y
                            #数値の整形[カンマ、小数点なし、右詰め]
                            ws.cell(j+1,col).number_format = "#,##0"
                            ws.cell(j+1,col).alignment = Alignment(horizontal='right')
                            break
        
    return wb

wb = honsya(file,data)

wb.save("本社2.xlsx")